# 0. Baseline HQNN Reproduction
This notebook reproduces the exact HQNN configuration used in the original paper, using a ZZ feature map, RealAmplitudes ansatz, and no noise.

In [2]:
# Setup and Imports
import sys
sys.path.append("..")

import torch
from torch import nn, optim
from hqnn_core.qnn_builder import create_qnn
from hqnn_core.classical_model import ClassicalNN
from hqnn_core.hqnn_model import HybridModel
from hqnn_core.train import train_model
from hqnn_core.evaluate import evaluate_model
from hqnn_core.data_utils import load_rssi_dataset

In [3]:
# Experiment Configs
scenarios = {
    "scenario_one": "1",
    "scenario_two": "2",
    "scenario_three": "3",
}
signals = ["bluetooth", "wifi", "zigbee"]

feature_map_type = "zz"
ansatz_type = "real"
num_qubits = 3
reps = 1
batch_size = 8
num_epochs = 30
learning_rate = 0.01

results = []

In [4]:
# Load Data and Prep Models for Configs
runs = []

for scenario, sc_num in scenarios.items():
    for signal in signals:
        print(f"Preparing {scenario.upper()} + {signal}...")

        train_loader, test_loader = load_rssi_dataset(
            scenario=scenario,
            signal=signal,
            sc_num=sc_num,
            batch_size=batch_size,
            base_path="../data",
        )

        qnn = create_qnn(
            num_qubits=num_qubits,
            reps=reps,
            feature_map_type=feature_map_type,
            ansatz_type=ansatz_type
        )
        clnn = ClassicalNN(input_size=3, hidden_size=32, output_size=2)
        model = HybridModel(qnn, clnn)

        runs.append((scenario, signal, model, train_loader, test_loader))


Preparing SCENARIO_ONE + bluetooth...
Preparing SCENARIO_ONE + wifi...
Preparing SCENARIO_ONE + zigbee...
Preparing SCENARIO_TWO + bluetooth...
Preparing SCENARIO_TWO + wifi...
Preparing SCENARIO_TWO + zigbee...
Preparing SCENARIO_THREE + bluetooth...
Preparing SCENARIO_THREE + wifi...
Preparing SCENARIO_THREE + zigbee...


In [5]:
# Model Training
for scenario, signal, model, train_loader, _ in runs:
    print(f"Training {scenario.upper()} + {signal}")
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    train_model(model, train_loader, criterion, optimizer, num_epochs=num_epochs, verbose=False)

Training SCENARIO_ONE + bluetooth
Training SCENARIO_ONE + wifi
Training SCENARIO_ONE + zigbee
Training SCENARIO_TWO + bluetooth
Training SCENARIO_TWO + wifi
Training SCENARIO_TWO + zigbee
Training SCENARIO_THREE + bluetooth
Training SCENARIO_THREE + wifi
Training SCENARIO_THREE + zigbee


In [6]:
# Model Evaluation
for scenario, signal, model, _, test_loader in runs:
    rmse = evaluate_model(model, test_loader, verbose=True)
    results.append((scenario, signal, rmse))

✅ RMSE on test set: 1.4366
✅ RMSE on test set: 1.4514
✅ RMSE on test set: 1.3100
✅ RMSE on test set: 1.4555
✅ RMSE on test set: 1.5366
✅ RMSE on test set: 0.8892
✅ RMSE on test set: 1.4559
✅ RMSE on test set: 1.1550
✅ RMSE on test set: 1.5032


In [7]:
# Results
import pandas as pd

results_df = pd.DataFrame(results, columns=["Scenario", "Signal", "RMSE"])
results_df.sort_values(by="RMSE", inplace=True)
display(results_df)

,Scenario,Signal,RMSE
5,scenario_two,zigbee,0.889173
7,scenario_three,wifi,1.154992
2,scenario_one,zigbee,1.310031
0,scenario_one,bluetooth,1.436647
1,scenario_one,wifi,1.451393
3,scenario_two,bluetooth,1.455492
6,scenario_three,bluetooth,1.455864
8,scenario_three,zigbee,1.503159
4,scenario_two,wifi,1.536619
